In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
# chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [5]:
url='https://www.aefi.it/en/calendario/?fair_name=&expo_center_uuid=&fair_type=&municipality=&fair_year=2022&month=&product_sector_uuid=#'

driver.get(url)
time.sleep(5)

soup=bs(driver.page_source,'lxml')
cards=soup.find_all('div',{'class':'fiere--card'})
lp=' '.join(soup.find('ul',{'class':'pagination float-left'}).find_all('li')[-1].text.split())#.find('a')['href']


n=1
brek=True
cards=[]
while brek:
    url=f'https://www.aefi.it/en/calendario/?fair_name=&expo_center_uuid=&fair_type=&municipality=&fair_year=2022&month=&product_sector_uuid=&page_nr={n}&page_size=20'
    driver.get(url)
    time.sleep(3)
    soup=bs(driver.page_source,'lxml')
    car=soup.find_all('div',{'class':'fiere--card'})
    
    for j in car:
        ritle=' '.join(j.find('div',{'class':'fiere--card_text'}).get_text(separator='{').split()).replace('{ { {','{').replace('{ {','{').split('{')[1].strip()

        link=j.find('a')['href']
   
        dat_dur=' '.join(j.find('div',{'class':'fiere--card_text'}).get_text(separator='{').split()).replace('{ { {','{').replace('{ {','{').split('{')[2].replace('dal','').replace('al','-').strip()
        
        cards.append([link,ritle,dat_dur])

    if n==int(lp):#lastpage
        brek=False
    n+=1


In [45]:
aefi=[]
for k in cards:
    link=k[0]
    driver.get(link)
    time.sleep(3)
    soupn=bs(driver.page_source,'lxml')
    if 'temporanly unavailable' in soupn.text:
        pass
    else:
        title=' '.join(soupn.find('div',{'class':'aefi--card_textbox title'}).h5.text.split())
        dat=' '.join(soupn.find('div',{'class':'aefi--card_textbox title'}).find('span',{'class':'date'}).text.split()).replace('dal','').replace('al','-').strip()

        dat=dat.replace('–','-').replace('|','').replace('to','-').replace('TO','-').replace('c-ber','ctober').replace('C-BER','CTOBER').strip()
        if '(' in dat:
            dat=dat.split('(')[0].strip()
        if dat=='':
            start_date=end_date=''
        else:
            mad=dat
            #try:
        #             ma=re.search('([A-Sa-z]+\W+\d{1,2}.+\W+\d{4}|\d{1,2}.+\W+\d{4})',_date)#, maxsplit=0)
        #             mad=ma.group()

            if '-' in mad:
                st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

            elif 'and'in mad:
                st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

            elif '&' in mad:
                st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

            else:
                st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            #print(date__,st,en)
            ####################
            if any(c.isalpha() for c in st)==False:
            #      print('leg')
                pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
                sapa=pa.group()
                start=st+' '+sapa
                end=en
            #                        02        April      2022
            elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
             #   print('awujale')
                start=st
                end=en

            #*                        April       02      2022
            elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
            #       print('ala')
                pa=re.search('(\d{4})',st).group()
                sa=re.search('([A-Sa-z]{3,9})',st).group()
                ta=re.search('(\d{1,2})',st).group()
                start=ta+' '+sa+' '+pa
                #
                ba=re.search('(\d{4})',en).group()
                ca=re.search('([A-Sa-z]{3,9})',en).group()
                da=re.search('(\d{1,2})',en).group()
                end=da+' '+ca+' '+ba     

            #                      02     2022
            elif re.search('\d{1,2}\s+\d{4}',en):
            #      print('kum')
                pa=re.search('(\d{4})',en).group()#, maxsplit=0)
                sa=re.search('([A-Sa-z]{3,9})',st).group()
                ta=re.search('(\d{1,2})',st).group()

                start=ta+' '+sa+' '+pa
                #
                ba=re.search('(\d{4})',en).group()
                ca=re.search('([A-Sa-z]{3,9})',st).group()
                da=re.search('(\d{1,2})',en).group()
                end=da+' '+ca+' '+ba

            #                      02         April
            elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
            #        print('is')
                pa=re.search('(\d{4})',en)#, maxsplit=0)
                sapa=pa.group()
                start=st+' '+sapa
                end=en

            #*                      April           02
            elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
            #      print('bad')
                pa=re.search('(\d{4})',en).group()#, maxsplit=0)
                sa=re.search('([A-Sa-z]{3,9})',st).group()
                ta=re.search('(\d{1,2})',st).group()

                start=ta+' '+sa+' '+pa
                #
                ba=re.search('(\d{4})',en).group()
                ca=re.search('([A-Sa-z]{3,9})',en).group()
                da=re.search('(\d{1,2})',en).group()
                end=da+' '+ca+' '+ba

            else:
            #     print('shik')
                start=end=''
            if start=='':
                start_date=end_date=''
            else:
                pick=[start, end]
                try:
                    spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
                except:
                    spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
                date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
                start_date=date_[0]
                end_date=date_[1]
        try:
            loc=' '.join(soupn.find('div',{'class':'aefi--card_textbox title'}).find('a').text.split())
        except:
            loc=''
        try: 
            cit=' '.join(soupn.find('div',{'class':'aefi--card_textbox title'}).get_text(separator='{').split()).split('{')[-1].strip()
        except:
            cit=''
        try:
            almond=' '.join(soupn.find('div',{'class':'dettaglio--fiere_address mt-3'}).find_all('address')[-1].get_text(separator='{').split()).replace('{ {','{')
            mai=almond.split('Email:')[-1].split('{')[1].strip()
            event_web=almond.split('Sito web:')[-1].split('{')[1].strip()    
            if mai=='':
                mai='info@aefi.it'
            if event_web=='':
                event_web=link
        except:
            mai='info@aefi.it'
            event_web=link
        
        def gu(luc):
            google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
            time.sleep(1)
            google_map_url=driver.current_url
    #                 print(google_map_url)
            return(google_map_url)
            ######################################
        def get_google_map_url(location):
            try:
                return(gu(location))
            except:
                try:
                    return(gu(location))
                except:
                    sha=location.split(',')
                    try:
                        gu(sha[-3])
                    except:
                        try:
                            gu(sha[-2])
                        except:
                            try:
                                gu(sha[-1])
                            except Exception as e:
                                print(location, "; url didn't go through")
                                print(e)
                                return("")


        def countr(locale):
            try:
                google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                time.sleep(randint(0,3))
                driver.get(google_url_for_location)
                time.sleep(4)
                try:
                    google_map_url=driver.find_element_by_id('lu_map').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                    except:
                        try:
                            google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                        except:
                            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                            time.sleep(randint(0,3))
                            driver.get(google_url_for_location)
                            time.sleep(4)
                            google_map_url=driver.find_element_by_id('lu_map').click()
                time.sleep(1)
                try:
                    country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
                except:
                    country=driver.find_element_by_class_name('bwoZTb').text
    #                 print(country)
                return(country)
            except Exception as e:
                print(locale, ";country didn't go through")
                print(e)
                return("")

        time_=''
        org_name='Associazione Esposizioni e Fiere italiane'
        org_web='https://www.aefi.it/en/'
        org_pro='AEFI – Associazione Esposizioni e Fiere Italiane (the Italian Exhibition and Trade Fair Association) represents the interests of the country’s trade fair industry: AEFI currently includes 47 members, with 41 exhibition districts, which hold over 1,000 events a year across an overall exhibition area of 4.2 million square metres, and 8 organisers only. On behalf of these associates, AEFI is taking action towards Parliament and the Government so that the Italian trade fair industry can better face international competition. Similar action is being taken at the Regional level, so that the transformation process currently underway may continue.'
        logo=''    
        sponsor=''
        agendalist=''
        type_=''
        category=''
        Speakerlist='' 
        ticket_list=''
        event_info=''
        
        
#         try:
        mail_=json.dumps([mai])
#         except:
#             mail_=''
        if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc:
            on_off='1'
            city=''
            venue=''
            country=''
            googlePlaceUrl=''
        else:
            on_off='0'
            city=cit
            country=countr(city)
            venue=loc
            googlePlaceUrl=get_google_map_url(venue)


        aefi.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                     org_pro,org_name,org_web,logo,sponsor,agendalist,
                     type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                     Speakerlist,on_off])


Verona ;country didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="lu_map"]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A46463+2188387]
	Ordinal0 [0x009DE461+1762401]
	Ordinal0 [0x008F3D78+802168]
	Ordinal0 [0x00921880+989312]
	Ordinal0 [0x00921B1B+989979]
	Ordinal0 [0x0094E912+1173778]
	Ordinal0 [0x0093C824+1099812]
	Ordinal0 [0x0094CC22+1166370]
	Ordinal0 [0x0093C5F6+1099254]
	Ordinal0 [0x00916BE0+945120]
	Ordinal0 [0x00917AD6+948950]
	GetHandleVerifier [0x00CE71F2+2712546]
	GetHandleVerifier [0x00CD886D+2652765]
	GetHandleVerifier [0x00AD002A+520730]
	GetHandleVerifier [0x00ACEE06+516086]
	Ordinal0 [0x009E468B+1787531]
	Ordinal0 [0x009E8E88+1805960]
	Ordinal0 [0x009E8F75+1806197]
	Ordinal0 [0x009F1DF1+1842673]
	BaseThreadInitThunk [0x7714FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77AC7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77AC7A6E+238]

VERONAFIERE S.P.A. ; url didn't go thr

Arezzo ;country didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="lu_map"]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A46463+2188387]
	Ordinal0 [0x009DE461+1762401]
	Ordinal0 [0x008F3D78+802168]
	Ordinal0 [0x00921880+989312]
	Ordinal0 [0x00921B1B+989979]
	Ordinal0 [0x0094E912+1173778]
	Ordinal0 [0x0093C824+1099812]
	Ordinal0 [0x0094CC22+1166370]
	Ordinal0 [0x0093C5F6+1099254]
	Ordinal0 [0x00916BE0+945120]
	Ordinal0 [0x00917AD6+948950]
	GetHandleVerifier [0x00CE71F2+2712546]
	GetHandleVerifier [0x00CD886D+2652765]
	GetHandleVerifier [0x00AD002A+520730]
	GetHandleVerifier [0x00ACEE06+516086]
	Ordinal0 [0x009E468B+1787531]
	Ordinal0 [0x009E8E88+1805960]
	Ordinal0 [0x009E8F75+1806197]
	Ordinal0 [0x009F1DF1+1842673]
	BaseThreadInitThunk [0x7714FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77AC7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77AC7A6E+238]



In [ ]:
len(aefi)

In [ ]:
aefi

In [ ]:
aefi_dict=aefi


aefi_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=aefi_dict)
aefi_df.to_csv("aefi.tsv", sep = '\t',index=False)
print('Dusted')

In [ ]:
# driver.switch_to.window(driver.window_handles[0])

In [21]:
# s=1
# for k in cards:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[0])
#     s+=1
# print(s+1)

42


In [ ]:
# s=1
# for k in cards:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[0])
#     s+=1
# print(s+1)

In [ ]:
# for k in range(1,31):
#     time.sleep(1)
#     driver.switch_to.window(driver.window_handles[k])
